In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import io
tqdm.pandas()

In [2]:
prefix = 'slim_'
colnames=['timestamp', 'entity_id', 'entity_value', 'activity_annotation'] 
df = pd.read_csv("./data/data", names=colnames, sep="\t")
df['timestamp'] = pd.to_datetime(df['timestamp'], format='ISO8601', utc=True)
df = df.sort_values(by=['timestamp'])
df['time_from_last_event'] = df['timestamp'].diff().dt.seconds
df['time_from_last_event'] = df['time_from_last_event'].fillna(0)

In [3]:
df

,timestamp,entity_id,entity_value,activity_annotation,time_from_last_event
0,2009-06-10 00:00:00.024668+00:00,T003,19,NaN,0.0
1,2009-06-10 00:00:46.069471+00:00,T005,18.5,NaN,46.0
2,2009-06-10 00:00:47.047655+00:00,T003,18.5,NaN,0.0
3,2009-06-10 00:01:17.070215+00:00,T005,18,NaN,30.0
4,2009-06-10 00:01:18.036049+00:00,T004,19.5,NaN,0.0
...,...,...,...,...,...
726529,2009-08-05 23:44:43.054933+00:00,T001,24,NaN,16.0
726530,2009-08-05 23:44:59.058871+00:00,T001,23.5,NaN,16.0
726531,2009-08-05 23:45:15.047153+00:00,T001,24,NaN,15.0
726532,2009-08-05 23:50:00.062322+00:00,T001,23.5,NaN,285.0


In [4]:
def map_activity_name_to_type(act_name:str) -> str:
    return act_name.replace(' begin', '').replace(' end', '')

In [5]:
activities_df = df[~df['activity_annotation'].isnull()]
# find unique activities
activity_entity_names = df['activity_annotation'].dropna().unique().tolist()
# remove the begin/end remarks, they will be come entity values
activity_entity_type = [map_activity_name_to_type(entity_name) for entity_name in activity_entity_names]
# create an activity entity_id to name mapping
activity_entity_map = d = {s:f'A{str(i).zfill(3)}' for i, s in enumerate(activity_entity_type, 1)}

In [6]:
_tmp_act_df = pd.DataFrame()
_tmp_act_df['entity_id'] = activities_df['activity_annotation'].map(lambda act_name: activity_entity_map[map_activity_name_to_type(act_name)])
_tmp_act_df['timestamp'] = activities_df['timestamp']
_tmp_act_df['time_from_last_event'] = 0
_tmp_act_df['entity_value'] = activities_df['activity_annotation'].map(lambda act_name: 'begin' if act_name.endswith('begin') else 'end')

In [7]:
merged_act_sensor_df = pd.concat([df.drop('activity_annotation', axis=1), _tmp_act_df], ignore_index=True)
merged_act_sensor_df['timestamp'] = merged_act_sensor_df['timestamp']
merged_act_sensor_df['entity_id'] = merged_act_sensor_df['entity_id'].astype(str)
merged_act_sensor_df.sort_values(by='timestamp')
# factorize the categorical features
temp_sensor_mask = merged_act_sensor_df['entity_id'].str.startswith('T')
merged_act_sensor_df['sensor_change'] = ~merged_act_sensor_df['entity_id'].str.startswith('A')
mapped_categories, state_dict = pd.factorize(merged_act_sensor_df['entity_value'][~temp_sensor_mask])
mapped_entities, entity_dict = pd.factorize(merged_act_sensor_df['entity_id'])
merged_act_sensor_df.loc[~temp_sensor_mask, 'entity_value'] = mapped_categories
merged_act_sensor_df['entity_id'] = mapped_entities
merged_act_sensor_df['entity_value'] = merged_act_sensor_df['entity_value'].astype(str)

In [8]:
print(f'total {len(state_dict)}: {state_dict}')
print(f'total {len(entity_dict)}: {entity_dict}')
merged_act_sensor_df.sample(10)

total 4: Index(['ON', 'OFF', 'begin', 'end'], dtype='object')
total 45: Index(['T003', 'T005', 'T004', 'T001', 'T002', 'M005', 'M006', 'M002', 'M009',
       'M011', 'M012', 'M022', 'M008', 'M007', 'M003', 'M010', 'M014', 'M015',
       'M023', 'M001', 'M024', 'M021', 'M016', 'M018', 'M020', 'M019', 'M013',
       'M025', 'M027', 'M017', 'M004', 'M026', 'A002', 'A004', 'A006', 'A008',
       'A010', 'A012', 'A014', 'A016', 'A018', 'A020', 'A022', 'A024', 'A026'],
      dtype='object')


,timestamp,entity_id,entity_value,time_from_last_event,sensor_change
37395,2009-06-12 21:24:32.044969+00:00,5,0,1.0,True
517492,2009-07-20 12:40:28.043732+00:00,20,1,6.0,True
293882,2009-06-29 16:44:06.016766+00:00,2,23.5,6.0,True
39817,2009-06-13 07:17:38.005185+00:00,21,1,2.0,True
510546,2009-07-19 19:34:39.069641+00:00,21,1,1.0,True
382643,2009-07-08 18:59:01.096576+00:00,13,0,0.0,True
461108,2009-07-15 23:20:40.076524+00:00,1,22,1.0,True
177697,2009-06-23 07:56:04.009882+00:00,11,1,0.0,True
301813,2009-06-30 08:00:48.076235+00:00,6,0,0.0,True
199313,2009-06-23 21:02:55.045129+00:00,26,0,0.0,True


In [9]:
# save the dataframe
merged_act_sensor_df.to_parquet(f'./data/vae/_merged_act_sensor_df.parquet')

# create pivoted table


In [10]:
# unique entities
unique_entities = merged_act_sensor_df['entity_id'].unique()

print(f'{len(unique_entities)} unique entities: {unique_entities}')

45 unique entities: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]


In [11]:
first_values = initial_values = merged_act_sensor_df.groupby('entity_id').first().sort_values(by='timestamp').reset_index()
# create default values for each sensor before they receive their first update
# for temp sensors, the init values are their first values
# for Movement sensors, the init values are opposite of their first values, which is OFF
# for Act type, the init values are opposite of their first values, which is end
def map_init_value(x):
    try:
        float(x)
        return x
    except ValueError:
        if x == str(state_dict.get_loc('ON')):
            return str(state_dict.get_loc('OFF'))
        elif x == str(state_dict.get_loc('begin')):
            return str(state_dict.get_loc('end'))
    return x

initial_frame = pd.DataFrame()
initial_frame['entity_id'] = first_values['entity_id']
initial_frame['entity_value'] = first_values['entity_value'].map(map_init_value)
initial_frame

,entity_id,entity_value
0,0,19
1,1,18.5
2,2,19.5
3,3,21.5
4,4,21.5
5,5,0
6,32,2
7,6,0
8,7,0
9,8,0


In [12]:
# Pivot the dataframe so each entity_id becomes a column, and entity_value are the values
pivot_df = merged_act_sensor_df.pivot(index='timestamp', columns='entity_id', values='entity_value')
# Reset the index to have 'timestamp' as a column again
snapshot_df = pivot_df.reset_index()
# Prepend the initial states to the snapshot df
initial_state = initial_frame.set_index('entity_id').T
initial_state['timestamp'] = merged_act_sensor_df['timestamp'].min() - pd.Timedelta(seconds=1)  # Assume initial timestamp before first event

# prefix the dummy initial state to the dataset, offset by 1 second before the actual head of the events
snapshot_df = pd.concat([initial_state, snapshot_df], ignore_index=True)
# Forward-fill to propagate the last known state for each entity over time
snapshot_df = snapshot_df.ffill()

snapshot_df = snapshot_df.merge(merged_act_sensor_df, on='timestamp')
snapshot_df.rename(columns={'entity_id': 'changed_entity_id', 'entity_value': 'changed_entity_value'}, inplace=True)

# current timestamp hints
snapshot_df['second'] = snapshot_df['timestamp'].dt.second
snapshot_df['minute'] = snapshot_df['timestamp'].dt.minute
snapshot_df['hour'] = snapshot_df['timestamp'].dt.hour
snapshot_df['dayofweek'] = snapshot_df['timestamp'].dt.day
snapshot_df['weekofmonth'] = (snapshot_df['timestamp'].dt.day - 1) // 7 + 1
snapshot_df['monthofyear'] = snapshot_df['timestamp'].dt.month

# drop the first row as it's dummy initial values
snapshot_df = snapshot_df.drop(index=0)

In [13]:
snapshot_df.shape

(727733, 56)

In [14]:
snapshot_df.sample(50)

,0,1,2,3,4,5,32,6,7,8,...,changed_entity_id,changed_entity_value,time_from_last_event,sensor_change,second,minute,hour,dayofweek,weekofmonth,monthofyear
248467,17.5,17,18.5,22.5,22.5,1,3,1,1,1,...,26,1,4.0,True,54,29,5,26,4,6
356277,23,22.5,24.5,24.5,24.5,1,3,0,0,1,...,6,0,0.0,True,24,46,8,5,1,7
282436,23,23,23,24,24.5,1,3,1,1,1,...,1,23,63.0,True,53,22,14,28,4,6
716285,24,24,25.5,23.5,25.5,0,3,1,1,0,...,15,1,0.0,True,52,3,7,5,1,8
346734,24.5,24.5,25,25,24.5,1,3,1,1,1,...,20,0,3.0,True,33,18,12,4,1,7
487579,24.5,25,25,24.5,25.5,1,3,1,1,1,...,26,0,7.0,True,21,39,14,17,3,7
447664,23.5,23.5,24,24.5,25,1,3,1,1,1,...,10,1,0.0,True,22,54,15,14,2,7
293771,23,23.5,23,24.5,25.5,1,3,1,1,1,...,19,1,5.0,True,18,57,15,29,5,6
85481,17.5,17,18.5,21,20,1,3,1,1,1,...,18,1,0.0,True,34,34,6,17,3,6
642588,21,21,21.5,23,23,1,3,1,1,1,...,9,1,0.0,True,23,24,7,31,5,7


In [15]:
activities_entries = snapshot_df[~snapshot_df['sensor_change']]

In [16]:
activities_entries.sample(10)

,0,1,2,3,4,5,32,6,7,8,...,changed_entity_id,changed_entity_value,time_from_last_event,sensor_change,second,minute,hour,dayofweek,weekofmonth,monthofyear
373462,18.5,19,23,21,21,1,3,1,1,1,...,37,3,0.0,False,47,49,6,8,2,7
558041,24.5,24.5,26,24,26,1,3,1,0,1,...,42,2,0.0,False,29,38,20,23,4,7
286864,17.5,17,18.5,22,21,1,3,1,1,1,...,32,3,0.0,False,54,8,4,29,5,6
423486,23,23.5,23.5,23.5,23.5,1,3,0,1,1,...,43,2,0.0,False,27,20,10,12,2,7
725842,28.5,29,29,31,30,1,3,1,1,1,...,40,3,0.0,False,17,46,18,5,1,8
500602,25,26,25.5,26.5,26.5,1,3,1,1,1,...,38,3,0.0,False,50,56,17,18,3,7
608433,23.5,23.5,24,24.5,24.5,1,3,1,1,1,...,37,2,0.0,False,48,29,8,28,4,7
431010,24.5,24.5,25,24,26,1,3,1,1,1,...,42,3,0.0,False,43,33,21,12,2,7
222082,22.5,22,23.5,23.5,24,1,3,1,1,1,...,36,2,0.0,False,10,25,7,25,4,6
633778,24.5,25,24.5,23.5,24,1,3,1,1,1,...,38,3,0.0,False,43,15,10,30,5,7


In [21]:
activity_examples_count = len(activities_entries)
print(f'Total {activity_examples_count} positive examples')

Total 1200 positive examples


In [23]:
sensor_examples_count = 3 * activity_examples_count
sensor_entries = snapshot_df[snapshot_df['sensor_change']].sample(sensor_examples_count)
print(f'Total {len(sensor_entries)} positive examples')

Total 3600 positive examples


In [41]:
combined_sampled_df = pd.concat([sensor_entries, activities_entries])
combined_sampled_df['sensor_change'], uniques = pd.factorize(combined_sampled_df['sensor_change'])
combined_sampled_df = combined_sampled_df.drop(['timestamp', 'changed_entity_id', 'changed_entity_value'], axis=1)

In [35]:
uniques

Index([True, False], dtype='bool')

In [42]:
combined_sampled_df.sample(20)

,0,1,2,3,4,5,32,6,7,8,...,43,44,time_from_last_event,sensor_change,second,minute,hour,dayofweek,weekofmonth,monthofyear
587692,23.5,23,24,27.5,25.5,1,3,1,1,1,...,3,3,15.0,0,1,30,17,26,4,7
412848,22.5,22.5,22.5,23,23.5,1,3,1,1,1,...,3,3,0.0,0,13,13,12,11,2,7
63320,20.5,20,21,23,22,1,2,0,1,1,...,3,2,0.0,1,23,27,3,15,3,6
656562,26.5,26.5,28.5,24,26.5,1,3,1,1,1,...,3,3,0.0,1,12,5,20,31,5,7
47239,24,24,24.5,25,25.5,1,3,1,1,1,...,3,2,1.0,0,30,48,15,13,2,6
44402,23,24,24,23,23.5,1,3,1,1,1,...,3,2,0.0,0,14,30,11,13,2,6
36608,23,22.5,23.5,24.5,25,1,3,1,1,1,...,3,2,0.0,0,14,29,20,12,2,6
110502,18.5,18,19.5,21,20.5,1,3,0,0,1,...,3,3,0.0,0,34,29,9,19,3,6
615419,22.5,22,23.5,22.5,24.5,1,3,1,1,0,...,3,3,0.0,1,6,22,2,29,5,7
120946,19.5,19,22,20.5,20.5,1,3,1,1,1,...,3,3,0.0,1,43,0,8,20,3,6


In [43]:
combined_sampled_df.columns

Index([                     0,                      1,                      2,
                            3,                      4,                      5,
                           32,                      6,                      7,
                            8,                      9,                     10,
                           11,                     33,                     12,
                           13,                     14,                     15,
                           16,                     17,                     18,
                           34,                     19,                     20,
                           21,                     22,                     23,
                           24,                     25,                     35,
                           36,                     26,                     27,
                           28,                     37,                     29,
                           30,                     3

In [44]:
combined_sampled_df.to_parquet(f'./data/vae/combined_sampled_df.parquet')

/opt/conda/lib/python3.11/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
